In [ ]:
from nanopyx.data.download import ExampleDataManager

downloader = ExampleDataManager()

datasets = downloader.list_datasets()
print("Here is a list of example datasets:")
for i, label in enumerate(datasets):
    print(f"{i+1}) {label}")


In [ ]:
import os
from skimage.transform import resize
from skimage.io import imsave

# generate missing thumbnails

for dataset in datasets:
    info = downloader.get_dataset_info(dataset)
    path = downloader.get_thumbnail(dataset)
    if not os.path.isfile(path):
        print(f"generating thumbnail for {dataset}")
        # downloader.download_tiff_sequence(dataset)
        z = downloader.get_ZipTiffIterator(dataset)
        first_frame = z[0]
        z.close()
        # save first frame as a 64x64 8-bit thumbnail
        thumbnail = resize(first_frame, (64, 64), anti_aliasing=True).astype("float32")
        # normalize intensity to 0-255
        thumbnail = (thumbnail - thumbnail.min()) / (thumbnail.max() - thumbnail.min())
        thumbnail = (thumbnail * 255).astype("uint8")
        os.makedirs(os.path.split(path)[0], exist_ok=True)
        imsave(path, thumbnail)
        

In [ ]:

from skimage.io import imread
import seaborn as sns
import matplotlib.pyplot as plt

filenames = [downloader.get_thumbnail(label) for label in datasets]
labels = datasets

from skimage import data
import seaborn as sns
import matplotlib.pyplot as plt

# Create a figure with subplots arranged in a row
fig, axs = plt.subplots(1, len(filenames), figsize=(80, 64))

for i, filename in enumerate(filenames):
    image = imread(filename)
    info = downloader.get_dataset_info(labels[i])
    sns.heatmap(image, ax=axs[i], cmap='inferno', cbar=False, square=True, xticklabels=False, yticklabels=False)
    axs[i].set_title(labels[i])
    axs[i].set_xlabel(f"{info['data_shape']} {info['data_dtype']}\n{info['reference']}")

# Show the figure
plt.show()


In [ ]:
z = downloader.get_ZipTiffIterator("SMLMS2013_HDTubulinAlexa647", as_ndarray = True)

import stackview
stackview.slice(z, continuous_update=True, zoom_factor=4, zoom_spline_order=0)#, display_width=512, display_height=512)
